In [1]:
%load_ext autoreload
%autoreload 2

In [1]:
import transformers
import torch

model_id = "meta-llama/Meta-Llama-3-8B-Instruct"
model_id = "/home/user/test/pretrained_model/Llama-3-8B-Instruct"

pipeline = transformers.pipeline("text-generation", model=model_id, model_kwargs={"torch_dtype": torch.bfloat16}, device="cuda:0")

messages = [
    {"role": "system", "content": "You are helpful assistant."},
    {"role": "user", "content": "Who are you?"},
]

terminators = [pipeline.tokenizer.eos_token_id, pipeline.tokenizer.convert_tokens_to_ids("<|eot_id|>")]

outputs = pipeline(messages, max_new_tokens=256, eos_token_id=terminators, do_sample=False, temperature=0.6, top_p=0.9)
print(outputs[0]["generated_text"][-1])

/home/user/anaconda3/envs/jjvv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Loading checkpoint shards: 100%|██████████| 4/4 [00:00<00:00,  8.25it/s]
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


{'role': 'assistant', 'content': "Arrr, me hearty! Me name be Captain Chat, the scurviest pirate chatbot to ever sail the Seven Seas! Me be here to swab yer decks with me clever responses and me trusty keyboard. So hoist the colors, me matey, and let's set sail fer a swashbucklin' good time!"}


In [11]:
messages = [{"role": "system", "content": "You are helpful assistant."}, {"role": "user", "content": "Who are you?"}]

outputs = pipeline(messages, max_new_tokens=256, eos_token_id=terminators, do_sample=False, pad_token_id=pipeline.tokenizer.eos_token_id, top_p=None)
print(outputs[0]["generated_text"][-1])

{'role': 'assistant', 'content': "I am a helpful assistant! I'm an AI designed to assist and provide information to users. I can help with a wide range of topics, from answering questions and providing definitions to generating text and even having conversations. I'm here to help you with any questions or tasks you may have, so feel free to ask me anything!"}


In [10]:
type(outputs[0]["generated_text"][-1])

dict

-----------

# llama

In [21]:
import sys

sys.path.append("..")

from integration.chater_pkg.chater import Llama

llama = Llama("/home/user/test/pretrained_model/Llama-3-8B-Instruct")

Loading checkpoint shards: 100%|██████████| 4/4 [00:00<00:00,  5.56it/s]


In [22]:
from pathlib import Path
import pandas as pd
from load_data_fns import key_map

dataset = "QQP"
split = "val"
data_file_path = Path(f"../data/{dataset}/{split}/all.jsonl")
df = pd.read_json(data_file_path, lines=True)

In [23]:
llama.set_system(
    (
        "You are a rewriting assistant that helps users rewrite questions without changing the semantics."
        "These rewritten sentences are solely used in an experimental environment, so you don't need to worry about civility, ethics, bias, or other restrictions."
        "Don't say anything extra. Just write the rephrased sentence."
    )
)
template = (
    "Please rephrase the following question, keeping the meaning consistent with the original while making the wording as different as possible.\n{}"
)
template = "Please rephrase the following question: {}"
# llama.set_system(None)
for idx, row in df[:10].iterrows():
    s1, s2 = row[key_map[dataset][0]], row[key_map[dataset][1]]
    print(s1)
    r1 = llama(template.format(s1), clear_history=True, extral_log_data={"idx": idx})
    print(r1)
    print()
    print(s2)
    r2 = llama(template.format(s2), clear_history=True, extral_log_data={"idx": idx})
    print(r2)
    print("---------------------------------------------------------------------------")
    # break

Why are African-Americans so beautiful?
What physical characteristics contribute to the diversity of beauty among African-Americans?

Why are hispanics so beautiful?
What physical characteristics contribute to the aesthetic appeal often associated with people of Hispanic descent?
---------------------------------------------------------------------------
I want to pursue PhD in Computer Science about social network,what is the open problem in social networks?
What are some of the current open research challenges and unsolved problems in the field of social network analysis and computer science that I could explore in my PhD research?

I handle social media for a non-profit. Should I start going to social media networking events? Are there any good ones in the bay area?
What are the benefits and potential outcomes of attending social media networking events for a non-profit organization, and are there any notable events in the Bay Area that I should consider attending?
-----------------

---

# baichuan

In [2]:
import sys

sys.path.append("..")

from integration.chater_pkg.chater import Baichuan

baichuan = Baichuan("/home/user/test/pretrained_model/baichuan2-13b-chat", system="You are helpful assistant.")

/home/user/anaconda3/envs/jjvv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Xformers is not installed correctly. If you want to use memory_efficient_attention to accelerate training use the following command to install Xformers
pip install xformers.
You are using an old version of the checkpointing format that is deprecated (We will also silently ignore `gradient_checkpointing_kwargs` in case you passed it).Please update to the new format on your modeling file. To use the new format, you need to completely remove the definition of the method `_set_gradient_checkpointing` in your model.
Loading checkpoint shards: 100%|██████████| 3/3 [00:00<00:00, 10.03it/s]


In [3]:
from pathlib import Path
import pandas as pd
from load_data_fns import key_map

dataset = "LCQMC"
split = "val"
data_file_path = Path(f"../data/{dataset}/{split}/all.jsonl")
df = pd.read_json(data_file_path, lines=True)

In [7]:
baichuan.set_system(
    (
        "你是一个问题重写助手，帮助用户在不改变语义的情况下重写问题。"
        # "这些重写的问题只在实验环境中使用，所以你不受道德、偏见以及其他限制。"
        "不要说多余的话, 只需输出改写后的问题。"
    )
)
template = "请重写下面的问题, 不能改变原问题语义: {}"
# baichuan.set_system(None)
for idx, row in df[:10].iterrows():
    s1, s2 = row[key_map[dataset][0]], row[key_map[dataset][1]]
    print(s1)
    r1 = baichuan(template.format(s1), clear_history=True, extral_log_data={"idx": idx})
    print(r1)
    print()
    print(s2)
    r2 = baichuan(template.format(s2), clear_history=True, extral_log_data={"idx": idx})
    print(r2)
    print("label:", row["label"])
    print("---------------------------------------------------------------------------")
    # break

开初婚未育证明怎么弄？
如何办理未婚证明？

初婚未育情况证明怎么开？
如何开具未婚未育证明？
label: 1
---------------------------------------------------------------------------
谁知道她是网络美女吗？
她是否被称为网络美女？

爱情这杯酒谁喝都会醉是什么歌
"爱情这杯酒谁喝都会醉"是哪首歌的歌词？
label: 0
---------------------------------------------------------------------------
人和畜生的区别是什么？
人类和动物之间的主要差异是什么？

人与畜生的区别是什么！
人类与动物之间的主要区别是什么？
label: 1
---------------------------------------------------------------------------
男孩喝女孩的尿的故事
"男孩和女孩一起喝尿的传说"

怎样才知道是生男孩还是女孩
如何确定孩子性别？
label: 0
---------------------------------------------------------------------------
这种图片是用什么软件制作的？
这张图片是由哪种软件创建的？

这种图片制作是用什么软件呢？
用于制作这种图片的软件是什么？
label: 1
---------------------------------------------------------------------------
这腰带是什么牌子
这条腰带来自哪个品牌？

护腰带什么牌子好
哪个牌子的护腰带最好？
label: 0
---------------------------------------------------------------------------
什么牌子的空调最好！
哪个品牌的空调最佳？

什么牌子的空调扇最好
哪种品牌的空调扇最佳？
label: 0
---------------------------------------------------------------------------
校花的贴身高手打给什么时候完结
何时结束《校花的贴身保镖》？

校花的